In [9]:
# Import packages
import pandas as pd
import numpy as np
import itertools

In [10]:
# Load Ibovespa daily data and work with data
df_ibovespa = pd.read_excel('Ibovespa - daily data.xlsx')
df_ibovespa = df_ibovespa.rename(columns={'Data':'Date','Abertura':'Open','Máxima':'High','Mínima':'Low','Fechamento':'Close'})
df_ibovespa = df_ibovespa.drop(columns='ULT')
df_ibovespa = df_ibovespa.sort_values(by='Date').reset_index()
df_ibovespa = df_ibovespa.drop(columns='index')

# Calculate returns
df_ibovespa['Return'] = df_ibovespa['Close'] / df_ibovespa['Close'].shift(1) - 1
df_ibovespa.head(2)

,Date,Open,High,Low,Close,Return
0,2002-07-03,10854.0,10868.0,10520.0,10635.0,NaN
1,2002-07-04,10661.0,10721.0,10593.0,10655.0,0.001881


## Functions

In [11]:
# Function that receives the dataframe and the SMA period as an input and outputs simple moving averages (SMA) and buy/sell signals

def sma_signals(df,period):
    # Calculate simple moving averages (SMA)
    df['SMA'] = df['Close'].rolling(period).mean()
    df['SMA'] = df['SMA'].shift(1)
    df['SMA'] = df['SMA'].round(0)

    # Calculate signal
    df['Signal'] = np.where( (df['Open'] < df['SMA']) & (df['Close'] > df['SMA']),
                                    1,
                                    np.where( (df['Open'] > df['SMA']) & (df['Close'] < df['SMA']),-1,0)
                                    )
    df['Signal'] = df['Signal'].shift(1)
    df.loc[df_ibovespa['Signal'].isna(),'Signal'] = 0
    df.loc[df['Signal'].isna()] = 0

    return df

In [12]:
# Function that calculates the returns given a holding period
def holding_period_returns(df,holding_period):
    
    # Remove overlapping signals within holding period
    for i in range(0,len(df)):
        signal = df.iloc[i,7]
        if signal != 0:
            if i + holding_period + 2 >= len(df):
                break
            else:
                df.iloc[i+1:i + holding_period,7] = 0
    
    # Calculate trade returns
    df['Exit Date'] = np.where(df['Signal']!=0,df['Date'].shift(-holding_period),np.nan)
    df['Exit Price'] = np.where(df['Signal']!=0,df['Close'].shift(-holding_period),np.nan)
    df['Trade'] = np.where(df['Signal'] == -1,
                                    df['Open'] / df['Exit Price'] - 1,
                                    np.where(df['Signal'] == 1,df['Exit Price'] / df['Open'] - 1,0)
                                    )
    return df

In [13]:
# Function usage example
df = holding_period_returns(sma_signals(df_ibovespa,200),51)
df.head(2)

,Date,Open,High,Low,Close,Return,SMA,Signal,Exit Date,Exit Price,Trade
0,2002-07-03 00:00:00,10854.0,10868.0,10520.0,10635.0,NaN,NaN,0.0,NaT,NaN,0.0
1,2002-07-04 00:00:00,10661.0,10721.0,10593.0,10655.0,0.001881,NaN,0.0,NaT,NaN,0.0


In [14]:
# Run strategy for several combinations of SMA and holding periods
smas = range(5,252,5)
holding_periods = range(5,252,5)
df_results = pd.DataFrame(columns=['Date','Signal','Trade','SMA','HP'])
combinations = itertools.product(smas,holding_periods)
for sma, hp in combinations:
        df = holding_period_returns(sma_signals(df_ibovespa,sma),hp)
        df['SMA_periods'] = sma
        df['HP'] = hp
        df_results = pd.concat([df,df_results])
        del(df)

In [15]:
# Save results
df_results['Signal'] = [np.nan if x == 0 else x for x in df_results['Signal']]
df_results.to_csv('results.csv')